비행기 게임 알고리즘 


1. 게임초기화


2. 게임창 크기/제목 선택


3. 게임 내 필요한 설정
    
    3-1. 클라스를 활용해 표현함수 만들기
    3-2. 충돌함수 만들기
    3-3. 비행기 관련 설정
    3-4. 배경 관련 설정
    
    

4. 메인 이벤트

    4-1. FPS 설정
    4-2. 각종 입력 감지: 방향키, 종료버튼 
    4-3. 입력/시간에 따른 변화
        4-3-1. 방향키 설정
        4-3-2. 발사한 미사일 리스트로 만들기
        4-3-3. 화면 밖에 나간 미사일 삭제
        4-3-4. 무작위 적 비행선 생성
        4-3-5. 화면 밖에 나간 적 비행선 삭제
        4-3-6. 충돌에 따른 변화
    4-4. 이미지 그리기
    4-5. 이벤트 업데이트



5. 게임 종료

In [1]:
import pygame
import random


# 1. 게임 초기화
pygame.init()




# 2. 게임창 크기/제목 설정
size = [740,370]
screen = pygame.display.set_mode(size)

title = 'My Game'
pygame.display.set_caption(title)




# 3. 게임 내 필요한 설정
clock = pygame.time.Clock()                                       # FPS
color = (0,0,0)                                                   # 배경색
k = 0                                                             # 미사일 발사횟수 설정을 위한 초기변수



left_go = False                                                   # 방향키
right_go = False
up_go = False
down_go = False
space_go = False

rocket_list = []                                                  # 발사한 로켓 리스트
ufo_list = []                                                     # 장애물 리스트
 



# 3-1. Class를 활용한 간편한 표현함수
class object:                                                     # 객체들을 간단하게 표현위해
    def __init__(self):                                           # 좌표 넣는 기능
        self.x = 0                                                # x좌표 표현
        self.y = 0                                                # y좌표 표현
    def put_image(self, address):                                 # 이미지 넣기
        self.img = pygame.image.load(address)                     # 특정 이미지 주소에서 꺼내 넣기
    def change_size(self, sx, sy):                                # 이미지 크기 바꾸는 함수
        self.img = pygame.transform.scale(self.img, (sx, sy))
        self.sx, self.sy = self.img.get_size()
    def show(self):                                               # 이미지 그리는 함수 + 위치
        screen.blit(self.img, (self.x, self.y))
        

        
        
        
        
        
# 3-2. 메인 비행기 관련 설정
plane = object()
plane.put_image('plane.png')
plane.change_size(50,30)




# 3-3. 배경 관련 설정
background = object()
background.put_image('background.png')




# 3-4. 충돌함수 설정
def crash(a, b):                                       # 충돌함수: 좌표를 기준으로 하기
    if (a.x-b.sx <= b.x) and (b.x <= a.x+a.sx):
        if (a.y-b.sy <= b.y) and (b.y <= a.y+a.sy):   
            return True
        else:
            return False
    else : 
        return False






# 4. 메인 이벤트
running = True
while running:                                         # 게임진행

    
    
    
    # 4-1. FPS 설정
    clock.tick(60)
    
    
    
    
    # 4-2. 각종 입력 감지  (종료,방향키)                    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:                  # 종료버튼 입력
            running = False
        if event.type == pygame.KEYDOWN:               # 방향키 입력
            if event.key == pygame.K_UP:
                up_go = True
            elif event.key == pygame.K_DOWN:
                down_go = True
            elif event.key == pygame.K_RIGHT:
                right_go = True
            elif event.key == pygame.K_LEFT:
                left_go = True
            elif event.key == pygame.K_SPACE:          # 스페이스바 감지
                space_go = True
               
        
        
        
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_UP:
                up_go = False
            elif event.key == pygame.K_DOWN:
                down_go = False
            elif event.key == pygame.K_RIGHT:
                right_go = False
            elif event.key == pygame.K_LEFT:
                left_go = False
            elif event.key == pygame.K_SPACE:          # 스페이스바 감지
                space_go = False                   
        
        
        
        
        
        
    # 4-3. 입력, 시간에 따른 변화
    
    # 4-3-1. 방향키 설정
    if left_go == True:          
        plane.x -= 5                                  # 꾹 누를때 계속이동           
        if plane.x < 0:                  
            plane.x = 0                               # 화면 밖으로 안 벗어나게
    elif right_go == True:
        plane.x += 5
        if plane.x > 720:
            plane.x = 720
    elif up_go == True:
        plane.y -= 5
        if plane.y < 0:
            plane.y = 0
    elif down_go == True:
        plane.y += 5
        if plane.y > 350:
            plane.y = 350
   




    # 4-3-2. 발사한 미사일 리스트에 추가
    if space_go == True and k % 10 == 0:             # 스페이스바 누를때, 미사일 발사, 10번이벤트 발생시 한번발사
        rocket = object()                            # 미사일 정의(위의 클라스 활용)
        rocket.put_image("rocket.png")
        rocket.change_size(30,30)
        rocket.x = plane.x                           # 미사일의 출발 좌표설정
        rocket.y = plane.y
        rocket_list.append(rocket)                   # 스페이스바 누를때마다 미사일 추가
    k += 1
        
        
        
        
        
    # 4-3-3. 화면 밖에 나간 미사일 리스트
    delete_list = []
    for i in range(len(rocket_list)):                # 리스트에서 발사한 미사일 꺼내서 움직이게하기
        rocket_num = rocket_list[i]
        rocket_num.x += 5
        if rocket_num.x > 740:                       # 미사일이 화면 밖에 나가면 데이터 삭제하기
            delete_list.append(i)  
    delete_list.reverse()
    for d in delete_list:                            # 화면 나간 미사일은 리스트   
        del rocket_list[d]                           # 화면 나간 미사일 리스트 삭제
    
    
    
    
    
    # 4-3-4. 무작위 적비행선 등장
    if random.random() > 0.98:                       # random함수(0~1사이 랜덤수 반환), 해당코드는 2%확률임
        ufo = object()
        ufo.put_image("ufo.png")
        ufo.change_size(30,30)
        ufo.x = 740                                  # 맨 오른쪽에서 시작이므로 x좌표는 740 고정
        ufo.y = random.randrange(0,350)              # 0~350 사이의 y좌표에서 랜덤으로 발생 
        ufo_list.append(ufo)
    
    
    

    
    # 4-3-5. 화면 밖에 나간 적비행선 삭제할 리스트
    delete_list2 = []
    for j in range(len(ufo_list)):             
        ufo_num = ufo_list[j]
        ufo_num.x -= 5
        if ufo_num.x < 0:                  
            delete_list2.append(j)
    delete_list.reverse()
    for e in delete_list2:                         
        del ufo_list[e]       
    
    
    
    
    # 4-3-6. 충돌시 삭제
    dr_list = []
    du_list = []
    
    for i in range(len(rocket_list)):
        for j in range(len(ufo_list)):
            r = rocket_list[i]
            u = ufo_list[j]
            if crash(r,u) == True:
                dr_list.append(i)
                du_list.append(j)
    
    dr_list = list(set(dr_list))
    du_list = list(set(du_list))
    dr_list.reverse()
    du_list.reverse()
    
    try:
        for i in dr_list:
            del rocket_list[i]
        for j in du_list:
            del ufo_list[j]
    
    except:
        pass
    
    
    
    
    
    
    # 4-4. 그리기
    screen.fill(color)                            # 배경화면 채우기
    background.show()
    plane.show()                                  # 비행기 이미지 (0,0)에 그리기
    for i in rocket_list:                         # 발사한 미사일 그리기
        i.show()
    for i in ufo_list:
        i.show()
    
    
    # 4-5. 업데이트
    pygame.display.flip()                         # 업데이트
    
    
    
    
# 5. 게임종료 
pygame.quit()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
